# SQL Over Anything with Spark

- Examples From Video Lecture 

In [1]:
import pyspark
from pyspark.sql import SparkSession

bucket = "e-drill"

spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
        .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.1.2,org.apache.spark:spark-avro_2.12:3.1.2")\
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
        .config("spark.hadoop.fs.s3a.access.key", "minio") \
        .config("spark.hadoop.fs.s3a.secret.key", "SU2orange!") \
        .config("spark.hadoop.fs.s3a.fast.upload", True) \
        .config("spark.hadoop.fs.s3a.path.style.access", True) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR") # Keeps the noise down!!!

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8117ffa1-f89e-4bf2-a0f3-5185020257b0;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.1.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
	found org.apache.spark#spark-avro_2.12;3.1.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 153ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.271 from central in [default]
	org.apache.hadoop#hadoop-aws;3.1.2 from central in [default]
	org.apache.spark#spark-avro_2.12;3.1.2 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   0   |   0   |   0   ||   4   |   0   |
	---------------------------------------------------------------------
:: retri

In [6]:
c = spark.read.csv("file:///home/jovyan/datasets/customers/customers.csv", 
                   inferSchema=True, header=True)
c.toPandas()
c.createOrReplaceTempView("customers") # now its an SQL table in Spark!
query = '''
SELECT  Email, Gender, State, `Months Customer`
FROM customers 
    WHERE State = 'NY'
'''
df = spark.sql(query)
df.show()

+--------------------+------+-----+---------------+
|               Email|Gender|State|Months Customer|
+--------------------+------+-----+---------------+
|  afresco@dayrep.com|     M|   NY|              1|
| cling@superrito.com|     F|   NY|              6|
|etasomthin@superr...|     M|   NY|             28|
|   jpoole@dayrep.com|     F|   NY|             12|
| ojouglad@einrot.com|     M|   NY|             36|
| rovlight@dayrep.com|     M|   NY|             42|
| sladd@superrito.com|     M|   NY|             10|
|titupp@superrito.com|     F|   NY|             42|
| tpani@superrito.com|     M|   NY|              1|
+--------------------+------+-----+---------------+



## Setup

- Put data in the right places!!!
- Run these cells to ensure you have the data for the examples

In [2]:
! pip install -q minio

In [3]:
from minio import Minio

# Make the minio bucket
client = Minio("minio:9000","minio","SU2orange!", secure=False)
not client.bucket_exists(bucket) and client.make_bucket(bucket)

# open the example(s)
gp = spark.read.json("file:///home/jovyan/datasets/json-samples/google-places.json")
c = spark.read.csv("file:///home/jovyan/datasets/customers/customers.csv", inferSchema=True, header=True)
s = spark.read.csv("file:///home/jovyan/datasets/customers/surveys.csv",inferSchema=True, header=True)
g = spark.read.csv("file:///home/jovyan/datasets/grades/*.tsv",inferSchema=False, header=False, sep="\t")
k = spark.read.csv("file:///home/jovyan/datasets/stocks/stocks.csv",inferSchema=True, header=True)

# Put the examples in minio
gp.write.mode("Overwrite").json(f"s3a://{bucket}/google-places.json")
c.write.mode("Overwrite").csv(f"s3a://{bucket}/customers.csv",header=True)
s.write.mode("Overwrite").csv(f"s3a://{bucket}/surveys.csv",header=True)
g.write.mode("Overwrite").csv(f"s3a://{bucket}/grades.csv",header=False, sep=",")
g.write.mode("Overwrite").parquet(f"s3a://{bucket}/grades.parquet")
k.write.mode("Overwrite").csv(f"s3a://{bucket}/stocks.csv",header=True,sep=",")

#put the examples in HDFS
gp.write.mode("Overwrite").json(f"hdfs://namenode/user/root/{bucket}/google-places.json")
c.write.mode("Overwrite").csv(f"hdfs://namenode/user/root/{bucket}/customers.csv",header=True)
s.write.mode("Overwrite").csv(f"hdfs://namenode/user/root/{bucket}/surveys.csv",header=True)
g.write.mode("Overwrite").csv(f"hdfs://namenode/user/root/{bucket}/grades.csv",header=False, sep=",")
g.write.mode("Overwrite").parquet(f"hdfs://namenode/user/root/{bucket}/grades.parquet")

## Spark SQL

- Spark Supports ANSI Standard SQL
- Once you register a DataFrame as a Temp View, you can query it like an SQL table.
- The output is a DataFrame, that you can maniuplate further with spark!

In [7]:
c.createOrReplaceTempView("customers")
s.createOrReplaceTempView("surveys")
c.printSchema()
s.printSchema()

root
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Last IP Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Total Orders: integer (nullable = true)
 |-- Total Purchased: integer (nullable = true)
 |-- Months Customer: integer (nullable = true)

root
 |-- Email: string (nullable = true)
 |-- Twitter Username: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Household Income: string (nullable = true)
 |-- Own Home: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Favorite Department: string (nullable = true)



In [8]:
query = '''
select c.Email, c.Gender, c.State, c.`Months Customer`, s.`Own Home`, s.`Household Income`
from customers c left join surveys s on 
        c.Email = s.Email
    where c.State = 'NY'
    and c.`Months Customer` > 5
    and s.`Own Home` is not null
'''
df = spark.sql(query)
df.show()
df.printSchema()

+--------------------+------+-----+---------------+--------+--------------------+
|               Email|Gender|State|Months Customer|Own Home|    Household Income|
+--------------------+------+-----+---------------+--------+--------------------+
|etasomthin@superr...|     M|   NY|             28|      No|               39000|
|   jpoole@dayrep.com|     F|   NY|             12|     Yes|Prefer not to Answer|
| ojouglad@einrot.com|     M|   NY|             36|      No|               65000|
| rovlight@dayrep.com|     M|   NY|             42|      No|               28000|
| sladd@superrito.com|     M|   NY|             10|     Yes|               52000|
+--------------------+------+-----+---------------+--------+--------------------+

root
 |-- Email: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Months Customer: integer (nullable = true)
 |-- Own Home: string (nullable = true)
 |-- Household Income: string (nullable = true)



In [9]:
df.explain()

== Physical Plan ==
*(2) Project [Email#524, Gender#525, State#528, Months Customer#531, Own Home#97, Household Income#96]
+- *(2) BroadcastHashJoin [Email#524], [Email#93], Inner, BuildRight, false
   :- *(2) Filter ((((isnotnull(State#528) AND isnotnull(Months Customer#531)) AND (State#528 = NY)) AND (Months Customer#531 > 5)) AND isnotnull(Email#524))
   :  +- FileScan csv [Email#524,Gender#525,State#528,Months Customer#531] Batched: false, DataFilters: [isnotnull(State#528), isnotnull(Months Customer#531), (State#528 = NY), (Months Customer#531 > 5..., Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/datasets/customers/customers.csv], PartitionFilters: [], PushedFilters: [IsNotNull(State), IsNotNull(Months Customer), EqualTo(State,NY), GreaterThan(Months Customer,5),..., ReadSchema: struct<Email:string,Gender:string,State:string,Months Customer:int>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [id=#365]
      +- *(1) Filter (is

In [ ]:
c.createOrReplaceTempView("customers")
s.createOrReplaceTempView("surveys")
c.printSchema()
s.printSchema()

In [13]:

query = '''
select c.Email, c.Gender, c.State, c.`Months Customer`, s.`Own Home`, 
    case s.`Household Income` 
        when 'Prefer not to Answer' then null 
        else cast(s.`Household Income` as DOUBLE) 
    end as Income
from customers c left join surveys s on 
        c.Email = s.Email
    where c.State = 'NY'
    and c.`Months Customer` > 5
    and s.`Own Home` is not null
'''
df = spark.sql(query)
df.filter("Gender = 'F'").show()


+-----------------+------+-----+---------------+--------+------+
|            Email|Gender|State|Months Customer|Own Home|Income|
+-----------------+------+-----+---------------+--------+------+
|jpoole@dayrep.com|     F|   NY|             12|     Yes|  null|
+-----------------+------+-----+---------------+--------+------+



In [ ]:
df.printSchema()
df.explain()
df.createOrReplaceTempView("v_ny_customer_analysis")

## Spark SQL works with Nested Data

In [ ]:
gp.printSchema()

In [10]:
gp.createOrReplaceTempView("googleplaces")
query = '''
select name, geometry.location.lat, geometry.location.lng, explode(types) as type from googleplaces
'''
spark.sql(query).show()

+--------------------+-----------------+------------------+-----------------+
|                name|              lat|               lng|             type|
+--------------------+-----------------+------------------+-----------------+
|            Syracuse|       43.0481221|-76.14742439999999|         locality|
|            Syracuse|       43.0481221|-76.14742439999999|        political|
|Crowne Plaza Syra...|       43.0476078|       -76.1417642|          lodging|
|Crowne Plaza Syra...|       43.0476078|       -76.1417642|point_of_interest|
|Crowne Plaza Syra...|       43.0476078|       -76.1417642|    establishment|
|  The Parkview Hotel|       43.0476157|        -76.140986|          lodging|
|  The Parkview Hotel|       43.0476157|        -76.140986|point_of_interest|
|  The Parkview Hotel|       43.0476157|        -76.140986|    establishment|
|Jefferson Clinton...|       43.0472894|-76.15385049999999|          lodging|
|Jefferson Clinton...|       43.0472894|-76.15385049999999|point

## Registering a UDF for use in Spark SQL


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import * 

@udf(returnType=StringType()) 
def upperCase(str):
    return str.upper()

spark.udf.register("upperCase", upperCase)

spark.sql("select Email, upperCase(Email), Gender from v_ny_customer_analysis").show()

## Listing views in Your Session

In [ ]:
spark.sql("show views").show()

# Big Data To Small Data

Using the `.toPandas()` function will materilaize the spark DataFrame as a Python Pandas dataframe. This allows us to use small-data features available in Python as our code is no longer executing on the cluster.

## Graphing Output

- A Spark DataFrame can be converted to a Python Pandas dataframe with the `toPandas()` function.
- Once its in Pandas you can plot with standard plotting libraries.
- https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html


In [ ]:
c.toPandas().plot.bar(x="Email", y="Total Purchased")

## IPywidgets

- IPy Widgets allows us to create interactive notebooks
- https://ipywidgets.readthedocs.io/en/latest/
- This is a great example of going from "Big Data" to "Small Data"
- As a rule, we don't want to produce small data (Pandas DataFrame) until we have mapped or reduced results.

In [ ]:
from IPython.display import display, HTML
from ipywidgets import interact, interact_manual
c = spark.read.csv("file:///home/jovyan/datasets/customers/customers.csv", inferSchema=True, header=True)

display(HTML("<h1>Customers By State</h1>"))
states = c.select("State").distinct().toPandas()["State"].values
states.sort()
@interact(state=states)
def main(state :str):
    rows = c.where( c.State == state).toPandas()
    display(rows)

In [ ]:
from IPython.display import display, HTML
from ipywidgets import interact, interact_manual, widgets

display(HTML("<h1>Customers By Total Purchased</h1>"))
rangewidget = widgets.IntRangeSlider(
    value=[10, 1000],
    min=0,
    max=5000,
    step=10,
    description='Total Purchased:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

c.createOrReplaceTempView("customers")

@interact_manual(purchased=rangewidget)
def main(purchased: int):
    query = f'select * from customers where `Total Purchased` between {purchased[0]} and {purchased[1]}'
    display(spark.sql(query).toPandas())
    

In [ ]:
s.printSchema()

In [ ]:
s.orderBy("Favorite Department", ascending = False).show()


In [ ]:
s.orderBy(s["Favorite Department"],ascending = False).show()

In [ ]:
s.filter( "`Favorite Department` LIKE 'Prefer%' " ).show()


In [ ]:
from pyspark.sql.functions import *
s.filter(col("Favorite Department").startswith("Prefer") ).show()

In [ ]:
s.createOrReplaceTempView("surveys")

In [ ]:
spark.sql("show tables").show()

In [ ]:
z =spark.sql("select * from surveys where `Favorite Department` like 'Prefer%'")

In [ ]:
y = s.filter(col("Favorite Department").startswith("Prefer") )
z =spark.sql("select * from surveys where `Favorite Department` like 'Prefer%'")

In [ ]:
y.explain()

In [ ]:
z.explain()

In [ ]:
query = '''

SELECT *
    FROM  surveys
    WHERE `Favorite Department` like 'Prefer%'

'''

spark.sql(query)

In [ ]:
#pyspark
spark.read.csv("s3a://....")

In [ ]:
spark.read.format("csv").load("s3a:///...")

In [ ]:
s.write.format("foo.bvar.avro").mode("errorifexists")\
    .option("sep","-").option("header",True).save("file:///foo.csv")

In [ ]:
gp.select("geometry.location.lat",gp.geometry.location.lng.alias("lng")  ).show()

In [ ]:
gp.select(col("geometry.location.lat").alias("g.l.lat"),gp.geometry.location.lng.alias("g.l.lng")  ).show()

In [ ]:
gp.select("geometry.location.lat",gp.geometry.location.lng ).toDF("g.l.lat","g.l.lng").show()

In [ ]:
from pyspark import SparkFiles

In [ ]:
spark.sparkContext.addFile("https://raw.githubusercontent.com/mafudge/datasets/master/crime/nys-crime-1960-2012.csv")
df = spark.read.option("inferSchema",True).option("header", True)\
    .csv(SparkFiles.get("nys-crime-1960-2012.csv"))
df.printSchema()

In [ ]:
SparkFiles.get("nys-crime-1960-2012.csv")

In [ ]:
from IPython.display import display, HTML
from ipywidgets import interact, interact_manual


@interact_manual(name="", age=(1,99), retired=False, hobbies=["Biking","Skiing"])
def onclick(name,age,retired,hobbies):
    display(HTML(f"<p>{name}, {age} {retired} {hobbies}"))
    
